In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import pmdarima as pm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMA

## Data Colectiva

In [2]:
# Reading Data
data = pd.read_excel('../raw_data/data_colec_full.xlsx', engine='openpyxl')
data.head()

,Unnamed: 0,status,insurance_type,disease,claim_type,provider_type,state,sex,age,country_id,amount,date_issue,age_range,h_type
0,4,PAGADO,COLECTIVO,Dolor Abdominal Y Pelvico,EMERGENCIA,CLINICA,Miranda,F,47,29,14,2018-09-02,40-49,Ambulatorio
1,5,PAGADO,COLECTIVO,Control General De Salud De Rutina De Subpobla...,REEMBOLSO,No Informado,Estado No Identificado,F,35,25,290,2018-09-03,30-39,Hospitalización
2,6,PAGADO,COLECTIVO,Quiste Folicular De La Piel Y Del Tejido Subcu...,REEMBOLSO,No Informado,Estado No Identificado,F,64,14,696,2018-09-03,60-69,Hospitalización
3,7,PAGADO,COLECTIVO,Control General De Salud De Rutina De Subpobla...,REEMBOLSO,No Informado,Estado No Identificado,F,48,14,35,2018-09-03,40-49,Hospitalización
4,9,PAGADO,COLECTIVO,"Fractura De La Pierna, Inclusive El Tobillo",REEMBOLSO,No Informado,Estado No Identificado,M,88,24,585,2018-09-03,Mayor a 70,Hospitalización


In [3]:
# Dropping extra index column
data = data.drop(columns= 'Unnamed: 0')


In [4]:
data.dtypes

status                    object
insurance_type            object
disease                   object
claim_type                object
provider_type             object
state                     object
sex                       object
age                        int64
country_id                 int64
amount                     int64
date_issue        datetime64[ns]
age_range                 object
h_type                    object
dtype: object

In [5]:
data.disease.value_counts()

Hipertension Arterial Y Sus Complicaciones                      12710
Bronquitis Aguda                                                 4206
Control General De Salud De Rutina De Subpoblaciones Definid     3996
Diarrea Y Gastroenteritis De Presunto Origen Infeccioso          3045
Lumbalgia                                                        2745
                                                                ...  
Sarcoma De Kaposi                                                   1
Tumor Benigno Del Oido Medio Y Del Sistema Respiratorio             1
Toxoplasmosis                                                       1
Hipertension Materna, No Especificada                               1
Tumor Maligno Del Paladar                                           1
Name: disease, Length: 524, dtype: int64

In [16]:
data['covid_claims'] = data.disease.map(lambda x: 1 if 'Covid' in x else 0)

In [7]:
data.covid_claims.value_counts()

0    109170
1      2645
Name: covid_claims, dtype: int64

In [8]:
data_daily = data.groupby('date_issue', as_index = False).agg({'amount': 'sum', 'covid_claims': 'sum'})
data_daily

,date_issue,amount,covid_claims
0,2018-09-02,14,0
1,2018-09-03,8716,0
2,2018-09-04,23504,0
3,2018-09-05,48112,0
4,2018-09-06,11508,0
...,...,...,...
889,2021-03-04,174996,15
890,2021-03-05,180932,23
891,2021-03-06,69308,17
892,2021-03-07,42822,11


In [9]:
data_weekly = data_daily.resample('W-Mon', on='date_issue').sum().reset_index().sort_values(by='date_issue')
data_weekly

,date_issue,amount,covid_claims
0,2018-09-03,8730,0
1,2018-09-10,148950,0
2,2018-09-17,118735,0
3,2018-09-24,150795,0
4,2018-10-01,40850,0
...,...,...,...
127,2021-02-08,1107180,96
128,2021-02-15,1048086,106
129,2021-02-22,1047364,94
130,2021-03-01,1328375,166


In [19]:
data['covid_claims'] = data.disease.map(lambda x: 1 if 'Covid' in x else 0)

In [26]:
data['covid_claims'].sum(data.amount)

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [10]:
# Exporting to excel

daily_data_clean_with_covid_col = data_daily.to_excel('../raw_data/daily_data_clean_with_covid_col.xlsx', engine='xlsxwriter')
weekly_data_clean_with_covid_col = data_weekly.to_excel('../raw_data/weekly_data_clean_with_covid_col.xlsx', engine='xlsxwriter')

## Data Individuos

In [11]:
# Reading Data
data = pd.read_excel('../raw_data/data_indiv_full.xlsx', engine='openpyxl')

In [12]:
data.head()

,Unnamed: 0,status,insurance_type,disease,claim_type,provider_type,state,sex,age,country_id,amount,date_issue,age_range,h_type
0,0,PAGADO,INDIVIDUAL,Laringitis Y Traqueitis Agudas,EMERGENCIA,CLINICA,Distrito Capital,F,48,29,26,2018-09-01,40-49,Ambulatorio
1,1,PAGADO,INDIVIDUAL,Hepatitis Viral(A-B-C),EMERGENCIA,CLINICA,Distrito Capital,F,50,29,142,2018-09-01,50-59,Ambulatorio
2,2,PAGADO,INDIVIDUAL,Amigdalo-Faringitis Aguda,EMERGENCIA,CLINICA,Zulia,M,42,29,21,2018-09-02,40-49,Ambulatorio
3,3,PAGADO,INDIVIDUAL,Bronquitis Aguda,EMERGENCIA,CLINICA,Distrito Capital,F,59,29,311,2018-09-02,50-59,Hospitalización
4,8,PAGADO,INDIVIDUAL,Gastritis (Simple),REEMBOLSO,No Informado,Estado No Identificado,M,26,29,18,2018-09-03,20-29,Hospitalización


In [13]:
# Dropping extra index column
data = data.drop(columns= 'Unnamed: 0')


In [14]:
data.dtypes

status                    object
insurance_type            object
disease                   object
claim_type                object
provider_type             object
state                     object
sex                       object
age                        int64
country_id                 int64
amount                     int64
date_issue        datetime64[ns]
age_range                 object
h_type                    object
dtype: object

In [15]:
data.disease.value_counts()

Embarazo, Parto Y Puerperio                                  2455
Hipertension Arterial Y Sus Complicaciones                   1835
Infeccion Viral - Covid 19 Sospechoso                        1582
Bronquitis Aguda                                             1200
Diarrea Y Gastroenteritis De Presunto Origen Infeccioso       967
                                                             ... 
Nistagmo Y Otros Movimientos Oculares Irregulares               1
Tumor Maligno Del Esofago                                       1
Convulsiones Del Recien Nacido                                  1
Quistes De La Region Bucal, No Clasificados En Otra Parte       1
Dermatopolimiositis                                             1
Name: disease, Length: 446, dtype: int64

In [15]:
data['covid_claims'] = data.disease.map(lambda x: 1 if 'Covid' in x else 0)

In [16]:
data.covid_claims.value_counts()

0    37537
1     1885
Name: covid_claims, dtype: int64

In [17]:
data_daily = data.groupby('date_issue', as_index = False).agg({'amount': 'sum', 'covid_claims': 'sum'})
data_daily

,date_issue,amount,covid_claims
0,2018-09-01,168,0
1,2018-09-02,332,0
2,2018-09-03,7453,0
3,2018-09-04,5025,0
4,2018-09-05,16023,0
...,...,...,...
914,2021-03-04,175724,27
915,2021-03-05,112495,22
916,2021-03-06,264350,21
917,2021-03-07,97002,15


In [1]:
amount = data_daily['covid_claims'].map(lambda x: sum(data_daily.amount) if data_daily['covid_claims'] > 0 else 0)

NameError: name 'data_daily' is not defined

In [19]:
data_weekly = data_daily.resample('W-Mon', on='date_issue').sum().reset_index().sort_values(by='date_issue')
data_weekly

,date_issue,amount,covid_claims
0,2018-09-03,7953,0
1,2018-09-10,52824,0
2,2018-09-17,144926,0
3,2018-09-24,334429,0
4,2018-10-01,202143,0
...,...,...,...
127,2021-02-08,745803,98
128,2021-02-15,677139,79
129,2021-02-22,1184838,113
130,2021-03-01,1130230,105


In [20]:
# Exporting to excel

daily_data_clean_with_covid_ind = data_daily.to_excel('../raw_data/daily_data_clean_with_covid_ind.xlsx', engine='xlsxwriter')
weekly_data_clean_with_covid_ind = data_weekly.to_excel('../raw_data/weekly_data_clean_with_covid_ind.xlsx', engine='xlsxwriter')